In [3]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray, compiler
from osgeo import gdal
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def cast_ray(point, direction, dsm, vegdem, vegdem2, scale):
    """Cast a ray from the point in the given direction and check for obstructions."""

    azimuth, altitude = direction
    dx = np.cos(np.radians(altitude)) * np.sin(np.radians(azimuth))
    dy = np.cos(np.radians(altitude)) * np.cos(np.radians(azimuth))
    dz = np.sin(np.radians(altitude))

    x, y = point
    z = dsm[int(y), int(x)]  # Start at DSM height

    while 0 <= x < dsm.shape[1] and 0 <= y < dsm.shape[0]:
        # Calculate the height of the surface at the current point
        dsm_height = dsm[int(y), int(x)]
        veg_height = vegdem[int(y), int(x)]
        canopy_bottom_height = vegdem2[int(y), int(x)]
        
        # Determine the height of the obstacle at this point
        obstacle_height = dsm_height + veg_height  # Height including vegetation
        
        # Check if the ray intersects with the obstacle
        if z < canopy_bottom_height:  # Under the canopy
            if z < dsm_height:  # If below DSM, then blocked
                return False
        elif z < obstacle_height:
            return False
        
        # Move to the next point in the direction of the ray
        x += dx * scale
        y += dy * scale
        z += dz * scale

    return True  # Ray is unobstructed

def svf_ray_tracing(dsm, vegdem, scale, num_azimuth_angles=360, num_altitude_angles=90):
    """Calculate the Sky View Factor using ray tracing."""

    rows, cols = dsm.shape
    svf = np.zeros((rows, cols))

    azimuth_angles = np.linspace(0, 360, num_azimuth_angles, endpoint=False)
    altitude_angles = np.linspace(0, 90, num_altitude_angles, endpoint=False)

    for y in range(rows):
        for x in range(cols):
            visible_count = 0
            total_count = num_azimuth_angles * num_altitude_angles

            for altitude in altitude_angles:
                for azimuth in azimuth_angles:
                    direction = (azimuth, altitude)
                    if cast_ray((x, y), direction, dsm, vegdem, scale):
                        visible_count += 1

            svf[y, x] = visible_count / total_count

    return svf

def svfCalculator_RayTracingOnGPU(dsm, vegdem, vegdem2, scale, transparency_factor):
    """Calculate the Sky View Factor on the GPU, incorporating vegetation transparency."""

    px = np.array(dsm).astype(np.float32)
    veg_px = np.array(vegdem).astype(np.float32)
    veg_px2 = np.array(vegdem2).astype(np.float32)
    height, width = px.shape
    
    # Allocate memory on the GPU
    d_px = gpuarray.to_gpu(px)
    d_veg_px = gpuarray.to_gpu(veg_px)
    d_veg_px2 = gpuarray.to_gpu(veg_px2)
    d_out = gpuarray.empty_like(d_px)
    
    # Define the CUDA kernel for SVF calculation
    kernel_code = """
    #include <math.h>
    #define PI 3.141592654
    __global__ void svfcalculator(float * lattice_out, float * lattice, float * veg_lattice, float * veg_lattice2, float scale, float transparency_factor, int width, int height)
    {
        int ix = threadIdx.x + blockIdx.x * blockDim.x;
        int iy = threadIdx.y + blockIdx.y * blockDim.y;
        if (ix >= width || iy >= height) return;
        int index = ix + iy * width;
        float SVF_res = 0;
        float clr00 = lattice[index];
        for (int thetaN = 0; thetaN < 720; thetaN++) 
        {
            float theta = PI * float(thetaN) / 180;
            float betaMax = 0;
            for (float radius = 1; radius < 450; radius += 1)
            {   
                int x = ix + int(radius * cos(theta));
                int y = iy - int(radius * sin(theta));
                if (x < 0 || x >= width || y < 0 || y >= height) break;
                int index2 = x + y * width;
                float dsm_height = lattice[index2];
                float veg_height = veg_lattice[index2];
                float canopy_bottom_height = veg_lattice2[index2];
                float combined_height = dsm_height + veg_height * transparency_factor;
                
                // Check if the ray is below the canopy bottom
                if (combined_height < canopy_bottom_height) {
                    combined_height = canopy_bottom_height;
                }
                
                float buildH = combined_height - clr00;
                float beta = atan(scale * buildH / radius); 
                betaMax = fmax(betaMax, beta);
            }
            SVF_res += pow(cos(betaMax), 2);
        }
        lattice_out[index] = SVF_res / 720.0;
    }
    """

    # Compile and execute the kernel
    mod = compiler.SourceModule(kernel_code)
    svfcalculator = mod.get_function("svfcalculator")

    # Thread and block dimensions for GPU execution
    nb_ThreadsX = 8
    nb_ThreadsY = 8
    nb_blocksX = (width + nb_ThreadsX - 1) // nb_ThreadsX
    nb_blocksY = (height + nb_ThreadsY - 1) // nb_ThreadsY
    
    # Execute the kernel on the GPU
    svfcalculator(
        d_out, d_px, d_veg_px, d_veg_px2, np.float32(scale), np.float32(transparency_factor),
        np.int32(width), np.int32(height),
        block=(nb_ThreadsX, nb_ThreadsY, 1),
        grid=(nb_blocksX, nb_blocksY)
    )
    
    return d_out.get()


def main(dsm_path, vegdem_path, vegdem2_path, scale, transparency_factor, output_path):
    """Main function to load data, calculate SVF, and save the result."""
    
    # Load DSM and vegetation DEM using GDAL
    dsm_ds = gdal.Open(dsm_path)
    vegdem_ds = gdal.Open(vegdem_path)
    vegdem2_ds = gdal.Open(vegdem2_path)  # Load the canopy bottom DEM
    
    dsm = dsm_ds.ReadAsArray().astype(np.float32)
    vegdem = vegdem_ds.ReadAsArray().astype(np.float32)
    vegdem2 = vegdem2_ds.ReadAsArray().astype(np.float32)  # Read the canopy bottom DEM
    
    # Apply Gaussian filter to vegdem
    vegdem_filtered = gaussian_filter(vegdem, sigma=0.5)
    
    # Calculate SVF using the GPU
    svf_result = svfCalculator_RayTracingOnGPU(dsm, vegdem_filtered, vegdem2, scale, transparency_factor)
    
    # Save the result as a GeoTIFF
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_path, dsm.shape[1], dsm.shape[0], 1, gdal.GDT_Float32)
    out_ds.GetRasterBand(1).WriteArray(svf_result)
    out_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    out_ds.SetProjection(dsm_ds.GetProjection())
    out_ds.FlushCache()
    

if __name__ == "__main__":
    # Define file paths and parameters
    dsm_path = "C:/Users/Max/Documents/Werk/test_gpu/input/dsm_150.tif"     # DSM raster file path
    vegdem_path = "C:/Users/Max/Documents/Werk/test_gpu/input/cdsm_150.tif"
    vegdem2_path = "C:/Users/Max/Documents/Werk/test_gpu/input/tdsm_150.tif"        # Vegetation DEM raster file path
    scale = 1.0  # Scale factor for distance calculation
    transparency_factor = 0.97  # Transparency factor for vegetation impact
    output_path = "C:/Users/Max/Documents/Werk/test_gpu/output/svf_37HN1_09_150.tif"  # Output file path

    gdal.DontUseExceptions()
    main(dsm_path, vegdem_path, vegdem2_path, scale, transparency_factor, output_path)
